## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
#pip install bootstrapped

In [2]:
import os
import shutil
import pandas as pd
import numpy as np
import glob
from scipy.stats import bootstrap

Get the AReM Data Set

In [3]:
dfs_test=[]  
dfs_train=[]   

#For bending1
filepath= 'AReM/bending1'
files=glob.glob(os.path.join(filepath,"*.csv"))

for i in files:
    if (i=="AReM/bending1/dataset1.csv") or (i=="AReM/bending1/dataset2.csv"):
        df=pd.read_csv(i, skiprows=4)
        dfs_test.append(df)
    else:
        df=pd.read_csv(i, skiprows=4)
        dfs_train.append(df)
        
#for bending2
filepath2= 'AReM/bending2'
files=glob.glob(os.path.join(filepath2,"*.csv"))

for i in files:
    if (i=="AReM/bending2/dataset1.csv") or (i=="AReM/bending2/dataset2.csv"):
        df=pd.read_csv(i,skiprows=4)
        dfs_test.append(df)
        
    elif (i=="AReM/bending2/dataset4.csv"):
        dfs=pd.read_csv(i,sep=' ',skiprows=5,names=["# Columns: time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"],index_col=False)
        dfs_train.append(dfs)
    else:
        df=pd.read_csv(i,skiprows=4)
        dfs_train.append(df)
        
#For cycling
file_cycle= 'AReM/cycling'
files=glob.glob(os.path.join(file_cycle,"*.csv"))

for i in files:
    if ((i=="AReM/cycling/dataset1.csv") or (i=="AReM/cycling/dataset2.csv") or(i=="AReM/cycling/dataset3.csv")):
        
        df=pd.read_csv(i,skiprows=4)
        dfs_test.append(df)
    else:
        dfs=pd.read_csv(i,skiprows=5, sep=' ',names=["# Columns: time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"],index_col=False)
        dfs_train.append(dfs)

        
#For lying dataset
file_lying= 'AReM/lying'
files=glob.glob(os.path.join(file_lying,"*.csv"))
 

for i in files:
    if ((i=="AReM/lying/dataset1.csv") or (i=="AReM/lying/dataset2.csv") or (i=="AReM/lying/dataset3.csv")):
        
        df=pd.read_csv(i,skiprows=4)
        
        dfs_test.append(df)
    else:
        df=pd.read_csv(i,skiprows=4)
        
        dfs_train.append(df)

#For sitting
file_sitting= 'AReM/sitting'
files=glob.glob(os.path.join(file_sitting,"*.csv"))
 

for i in files:
    if ((i=="AReM/sitting/dataset1.csv") or (i=="AReM/sitting/dataset2.csv") or (i=="AReM/sitting/dataset3.csv")):
        
        df=pd.read_csv(i ,skiprows=4)
        
        dfs_test.append(df)
    else:
        df=pd.read_csv(i ,skiprows=4)
        
        dfs_train.append(df)

#For standing
file_standing= 'AReM/standing'
files=glob.glob(os.path.join(file_standing,"*.csv"))
 

for i in files:
    if ((i=="AReM/standing/dataset1.csv") or (i=="AReM/standing/dataset2.csv") or (i=="AReM/standing/dataset3.csv")):
        df = pd.read_csv(i, skiprows=4)
        
        dfs_test.append(df)
    else:
        df=pd.read_csv(i, skiprows=4)
        
        dfs_train.append(df)
        
#For walking 
file_walking= 'AReM/walking'
files=glob.glob(os.path.join(file_walking,"*.csv"))
 

for i in files:
    if ((i=="AReM/walking/dataset1.csv") or (i=="AReM/walking/dataset2.csv") or (i=="AReM/walking/dataset3.csv")):
        
        df=pd.read_csv(i,skiprows=4)
        
        dfs_test.append(df)
    else:
        df=pd.read_csv(i,skiprows=4)
        
        dfs_train.append(df)             

dfs_test=pd.concat(dfs_test,axis=0,ignore_index=True)
dfs_train=pd.concat(dfs_train,axis=0,ignore_index=True)

### (b) Test and Train Data

In [4]:
dfs_test.head()

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.3
1,250,39.25,0.43,23.00,0.00,33.00,0.0
2,500,39.25,0.43,23.25,0.43,33.00,0.0
3,750,39.50,0.50,23.00,0.71,33.00,0.0
4,1000,39.50,0.50,24.00,0.00,33.00,0.0


In [5]:
dfs_test.shape

(9120, 7)

In [6]:
dfs_train.head()

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.00,18.50,0.50,12.00,0.00
1,250,42.00,0.00,18.00,0.00,11.33,0.94
2,500,42.75,0.43,16.75,1.79,18.25,0.43
3,750,42.50,0.50,16.75,0.83,19.00,1.22
4,1000,43.00,0.82,16.25,0.83,18.00,0.00


In [7]:
dfs_train.shape

(33119, 7)

### (c) Feature Extraction

#### i. Research

Time-domain features are often used in time series classification to extract useful information about the temporal characteristics of a signal. 

1. Min - Minimum 
2. Max - Maximum
3. Mean - the average value of the signal over time.
4. Standard deviation - a measure of the spread of the signal values.
5. Median - the middle value of the signal.
6. Interquartile range - the range between the 25th and 75th percentile of the signal.

#### ii. Extraction

In [8]:
coloums=[]
for k in range(1,7):
    coloums.append(['min'+str(k),'max'+str(k),'mean'+str(k),'median'+str(k),'std'+str(k),'1st quart'+str(k),'3rd quart'+str(k)])
fnl_coloumn = [item for sublist in coloums for item in sublist]

In [9]:
bending1=[]
for k in range(1,8):
    bending1.append("dataset"+str(k))
    
filepath_bend= 'AReM/bending1'
files=glob.glob(os.path.join(filepath_bend,"*.csv"))

df_final= pd.DataFrame(columns=[fnl_coloumn])
for i in bending1:
    df=pd.read_csv(filepath_bend+'/' + str(i)+".csv",skiprows=4)
    df.drop(columns=['# Columns: time'],axis=1)
    temp=[]
    for j in df.columns[1:]:
        temp.append(df[j].min())
        temp.append(df[j].max())
        temp.append(df[j].mean())
        temp.append(df[j].median())
        temp.append(df[j].std())
        temp.append(df[j].quantile(0.25))
        temp.append(df[j].quantile(0.75))
        
    df_final.loc[len(df_final)]=temp

In [10]:
bending2=[]
for k in range(1,7):
    bending2.append("dataset"+str(k))


filepath_bend2= 'AReM/bending2'
files=glob.glob(os.path.join(filepath_bend2,"*.csv"))

df_final2= pd.DataFrame(columns=[fnl_coloumn])

for i in bending2:
    temp2=[]
    if i == 'dataset4':
        path="AReM/bending2/"+str(i)+".csv"
        df_bend2=pd.read_csv(path,sep=' ',skiprows=5,names=["# Columns: time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"],index_col=False)  
        for j in df_bend2.columns[1:]:
            temp2.append(df_bend2[j].min())
            temp2.append(df_bend2[j].max())
            temp2.append(df_bend2[j].mean())
            temp2.append(df_bend2[j].median())
            temp2.append(df_bend2[j].std())
            temp2.append(df_bend2[j].quantile(0.25))
            temp2.append(df_bend2[j].quantile(0.75))
    else:
        df_bend2=pd.read_csv(filepath_bend2+'/' + str(i)+".csv",skiprows=4)
        for j in df_bend2.columns[1:]:
            temp2.append(df_bend2[j].min())
            temp2.append(df_bend2[j].max())
            temp2.append(df_bend2[j].mean())
            temp2.append(df_bend2[j].median())
            temp2.append(df_bend2[j].std())
            temp2.append(df_bend2[j].quantile(0.25))
            temp2.append(df_bend2[j].quantile(0.75))
            
    
    df_final2.loc[len(df_final2)]=temp2

In [11]:
cycling=[]
for k in range(1,16):
    cycling.append("dataset"+str(k))
    
filepath_cyc= 'AReM/cycling'
files=glob.glob(os.path.join(filepath_cyc,"*.csv"))


df_final_cyc= pd.DataFrame(columns=[fnl_coloumn])

for i in cycling:
    temp_cyc=[]
    
    if i!='dataset1'or i!='dataset2' or i!='dataset3':
        path="AReM/cycling/"+str(i)+".csv"
        df_cyc=pd.read_csv(path, sep=' ',skiprows=5, names=["# Columns: time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"],index_col=False)
        for j in df_cyc.columns[1:]:
            temp_cyc.append(df_cyc[j].min())
            temp_cyc.append(df_cyc[j].max())
            temp_cyc.append(df_cyc[j].mean())
            temp_cyc.append(df_cyc[j].median())
            temp_cyc.append(df_cyc[j].std())
            temp_cyc.append(df_cyc[j].quantile(0.25))
            temp_cyc.append(df_cyc[j].quantile(0.75))
    else:
        df_cyc=pd.read_csv(filepath_cyc+'/' + str(i)+".csv",skiprows=4)
        for j in df_cyc.columns[1:]:
            temp_cyc.append(df_cyc[j].min())
            temp_cyc.append(df_cyc[j].max())
            temp_cyc.append(df_cyc[j].mean())
            temp_cyc.append(df_cyc[j].median())
            temp_cyc.append(df_cyc[j].std())
            temp_cyc.append(df_cyc[j].quantile(0.25))
            temp_cyc.append(df_cyc[j].quantile(0.75))
    
        
       
    df_final_cyc.loc[len(df_final_cyc)]=temp_cyc
    

In [12]:
lying=[]
for k in range(1,16):
    lying.append("dataset"+str(k))

filepath_ly= 'AReM/lying'
files=glob.glob(os.path.join(filepath_ly,"*.csv"))

df_final_ly= pd.DataFrame(columns=[fnl_coloumn])
for i in cycling:
    df_ly=pd.read_csv(filepath_ly+'/' + str(i)+".csv",skiprows=4)
    temp_ly=[]
    for j in df_cyc.columns[1:]:
        temp_ly.append(df_ly[j].min())
        temp_ly.append(df_ly[j].max())
        temp_ly.append(df_ly[j].mean())
        temp_ly.append(df_ly[j].median())
        temp_ly.append(df_ly[j].std())
        temp_ly.append(df_ly[j].quantile(0.25))
        temp_ly.append(df_ly[j].quantile(0.75))
    
       
    df_final_ly.loc[len(df_final_ly)]=temp_ly

In [13]:
sitting=[]
for k in range(1,16):
    sitting.append("dataset"+str(k))

filepath_sit= 'AReM/sitting'
files=glob.glob(os.path.join(filepath_sit,"*.csv"))


df_final_sit= pd.DataFrame(columns=[fnl_coloumn])
for i in sitting:
    df_sit=pd.read_csv(filepath_sit+'/' + str(i)+".csv",skiprows=4)
    temp_sit=[]
    for j in df_sit.columns[1:]:
        temp_sit.append(df_sit[j].min())
        temp_sit.append(df_sit[j].max())
        temp_sit.append(df_sit[j].mean())
        temp_sit.append(df_sit[j].median())
        temp_sit.append(df_sit[j].std())
        temp_sit.append(df_sit[j].quantile(0.25))
        temp_sit.append(df_sit[j].quantile(0.75))
    
        
    df_final_sit.loc[len(df_final_sit)]=temp_sit

In [14]:
standing=[]
for k in range(1,16):
    standing.append("dataset"+str(k))

filepath_stan= 'AReM/standing'
files=glob.glob(os.path.join(filepath_stan,"*.csv"))


df_final_stan= pd.DataFrame(columns=[fnl_coloumn])
for i in standing:
    df_stan=pd.read_csv(filepath_stan+'/' + str(i)+".csv",skiprows=4)
    temp_stan=[]
    for j in df_stan.columns[1:]:
        temp_stan.append(df_stan[j].min())
        temp_stan.append(df_stan[j].max())
        temp_stan.append(df_stan[j].mean())
        temp_stan.append(df_stan[j].median())
        temp_stan.append(df_stan[j].std())
        temp_stan.append(df_stan[j].quantile(0.25))
        temp_stan.append(df_stan[j].quantile(0.75))
        
    
    df_final_stan.loc[len(df_final_stan)]=temp_stan

In [15]:
walking=[]
for k in range(1,16):
    walking.append("dataset"+str(k))

filepath_walk= 'AReM/walking'
files=glob.glob(os.path.join(filepath_walk,"*.csv"))


df_final_walk= pd.DataFrame(columns=[fnl_coloumn])
for i in walking:
    df_walk=pd.read_csv(filepath_walk+'/' + str(i)+".csv",skiprows=4)
    temp_walk=[]
    for j in df_walk.columns[1:]:
        temp_walk.append(df_walk[j].min())
        temp_walk.append(df_walk[j].max())
        temp_walk.append(df_walk[j].mean())
        temp_walk.append(df_walk[j].median())
        temp_walk.append(df_walk[j].std())
        temp_walk.append(df_walk[j].quantile(0.25))
        temp_walk.append(df_walk[j].quantile(0.75))
        
    
    df_final_walk.loc[len(df_final_walk)]=temp_walk

In [16]:
frames=[df_final,df_final2,df_final_cyc,df_final_ly,df_final_sit,df_final_stan,df_final_walk]
final_df= pd.concat(frames,ignore_index=True)

In [17]:
final_df

,min1,max1,mean1,median1,std1,1st quart1,3rd quart1,min2,max2,mean2,...,std5,1st quart5,3rd quart5,min6,max6,mean6,median6,std6,1st quart6,3rd quart6
0,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.43,0.582915,0.0000,1.3000
1,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,0.372438,...,1.995255,32.0000,34.50,0.00,3.11,0.571083,0.43,0.601010,0.0000,1.3000
2,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.00,1.79,0.493292,0.43,0.513506,0.0000,0.9400
3,33.00,47.75,42.179813,43.50,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.50,0.524317,0.0000,1.0000
4,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.43,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,19.50,45.33,33.586875,34.25,4.650935,30.25,37.0000,0.0,14.67,4.576562,...,3.283983,13.7300,18.25,0.00,8.32,3.259729,3.11,1.640243,2.0500,4.3225
84,19.75,45.50,34.322750,35.25,4.752477,31.00,38.0000,0.0,13.47,4.456333,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.20,1.732727,2.1575,4.5650
85,19.50,46.00,34.546229,35.25,4.842294,31.25,37.8125,0.0,12.47,4.371958,...,2.823124,14.0000,17.75,0.00,10.00,3.338125,3.08,1.656742,2.1600,4.3350
86,23.50,46.25,34.873229,35.25,4.531720,31.75,38.2500,0.0,14.82,4.380583,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.27,1.690960,2.1700,4.5000


#### iii. Standard Deviation

In [25]:
final_df.describe()

,min1,max1,mean1,median1,std1,1st quart1,3rd quart1,min2,max2,mean2,...,std5,1st quart5,3rd quart5,min6,max6,mean6,median6,std6,1st quart6,3rd quart6
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.0,72.000000,72.000000,...,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,30.028750,45.852500,39.494976,39.528681,2.851116,37.747049,41.625104,0.0,5.413750,1.219736,...,3.669400,13.436597,18.294826,0.005972,5.897500,1.338316,1.191181,0.910394,0.737500,1.764479
std,10.216682,4.590553,5.687615,5.763545,1.903009,6.559536,5.484257,0.0,5.029536,1.616355,...,1.071936,6.653618,6.066314,0.050676,2.405443,1.052155,1.009821,0.445189,0.723208,1.380138
min,0.000000,30.000000,24.562958,24.250000,0.032038,23.187500,26.500000,0.0,0.430000,0.007167,...,1.995255,1.000000,5.542500,0.000000,1.790000,0.383292,0.430000,0.388372,0.000000,0.500000
25%,21.500000,45.000000,34.586464,35.250000,1.440578,31.250000,38.000000,0.0,1.560000,0.363688,...,3.075863,10.421875,15.437500,0.000000,4.240000,0.698333,0.500000,0.608510,0.430000,0.947500
50%,33.290000,46.250000,42.145698,41.625000,2.389754,39.875000,43.000000,0.0,3.055000,0.445198,...,3.303720,13.615000,18.125000,0.000000,5.720000,0.862506,0.820000,0.737295,0.460000,1.120000
75%,37.062500,48.000000,43.521224,43.835000,4.671149,42.000000,45.000000,0.0,8.490000,0.585411,...,4.116093,14.812500,20.000000,0.000000,7.377500,1.156823,0.940000,1.039654,0.500000,1.350000
max,48.000000,56.250000,48.004167,48.000000,7.684146,48.000000,54.000000,0.0,17.240000,4.576562,...,7.853427,35.362500,36.500000,0.430000,13.610000,3.500750,3.285000,1.792090,2.240000,4.565000


In [19]:
stds = final_df.describe().loc["std"]
stds

min1          10.216682
max1           4.590553
mean1          5.687615
median1        5.763545
std1           1.903009
1st quart1     6.559536
3rd quart1     5.484257
min2           0.000000
max2           5.029536
mean2          1.616355
median2        1.480532
std2           0.857901
1st quart2     0.997428
3rd quart2     2.161815
min3           2.836313
max3           4.967620
mean3          4.229650
median3        4.307799
std3           1.039738
1st quart3     4.492750
3rd quart3     4.438369
min4           0.000000
max4           2.029274
mean4          1.046196
median4        1.033202
std4           0.388417
1st quart4     0.795318
3rd quart4     1.378237
min5           6.562029
max5           6.327583
mean5          6.209927
median5        6.360592
std5           1.071936
1st quart5     6.653618
3rd quart5     6.066314
min6           0.050676
max6           2.405443
mean6          1.052155
median6        1.009821
std6           0.445189
1st quart6     0.723208
3rd quart6     1

In [20]:
conf_interval=pd.DataFrame(columns=["The features","90% confidence interval"])
conf_intrvl=[]
conf_interval["The features"] =fnl_coloumn 

for i in range (0,len(final_df.columns)):
  # samples must be in a sequence
    res = bootstrap((final_df.iloc[:,i],), np.std, confidence_level=0.9,random_state=20, method='percentile')
    conf_intrvl.append(res.confidence_interval)

conf_interval["90% confidence interval"]=pd.Series(conf_intrvl) 

#### iv. Select Features

According to me the most important features are namely Standard Deviation, median, and mean. the most important features to seem like mean, median and standard deviation.

1) Standard Deviaiton because its a key features in determining the expansion of the curve and how it's shaped along the values of features. Also in our dataset, the range of confidence interval is the lowest for standard deviation and hence its an important features.

2) Median because its range of confidence interval is also among the smallest and median gives how skewed the distribution is,and represents the half of dataset and therefore gives a better perspective of the data. .

3) Mean because it gives the information about the position of peak of the distribution and the type of distribution and in our dataset its range for confidence interval is among the smallest ones.

## 2. ISLR 3.7.4
I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. (Y = β0 + β1X + β2X2 + β3X3 + e).

### (a) Linear Train

One would expect that the training RSS of the cubic regression to be lower than the RSS of the linear regression because the training RSS always goes down when the model flexibility goes up.

### (b) Linear Test

In this case the expected RSS of the linear regression should be lower because the cubic regression would have a higher variance, which would not be offsetted by a reduction in bias (since the “true model” is actually linear).

### (c) Not Linear Train

Same as (a). One would expect that the training RSS of the cubic regression to be lower than the RSS of the linear regression because the training RSS always goes down when the model flexibility goes up.

### (d) Not Linear Testing

It would depend on how non-linear is the true relationship between X and Y. If it’s just cuadratic, either the linear model or the cubic model could have a lower training RSS. However, if the true relationship is very far from linear, then one expect the cubic regression RSS to be lower.